<a href="https://colab.research.google.com/github/adm-2k/DataPPF/blob/main/HW5_S23_People_like_you_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5  
# "People like you..." (part 2)
<b>ASSIGNED: 10 April 2023.</b>   
<b>DUE: 25 April 2023 at 11:59 PM EST</b>


Be sure to rename this notebook so it includes your name.



#### Goal:

Building on homework 4, in this (final! short!) homework we're going to do the following:

1. Perform K-means to group individuals;
2. Speculate on the significance of one grouping created in this way;
3. Destroy privacy through de-anonymizing attacks.

The homework should take you about <b>2.5 hours</b> to do. Special care has been taken to give you detailed examples. You only need adapt the example for the question asked. If you need help, please ask for help early!


## Instructions
* This assignment is to be done <b>on your own</b>, but you can talk about the assignment with your classmates if you get stuck. (Be sure to list the students you spoke with about this assignment in the space provided below.)
* Feel free to also use [stackoverflow](https://stackoverflow.com/) but please provide citation and link to the specific answer if you do this.
* You may also visit Graders during office hours or message them on Slack with questions.
* <b>Only include relevant information in your output.</b> We should be able to see your answer clearly.
* <b>Save pdf in Landscape</b> and check that all of your code is shown in the submission.
* When submitting in Gradescope, be sure to <b>select the pages that corresponds to the code/output of the respective question</b>.


### List any students you talked with about this assignment here:
1. [person 1]
2. [person 2]
3. etc.



## Homework Problems

### Question 0: Stationkeeping Work [0 points]
First, let's load libraries and the data from homework 4:

In [ ]:
# loading libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib.axes as axes
from sklearn.cluster import KMeans
%matplotlib inline

In [ ]:
# load data again / you will need to be connected to the internet to get data
features = ['age','workclass', 'fnlwgt', 'education', 'education.num', 'marital.status','occupation', 'relationship','race','sex','capital.gain','capital.loss', 'hours.per.week', 'native.country','income']
data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', sep=',', header=None, na_values="?", names=features)

In [ ]:
# convert categorical data into numerical data
def preprocess_features(dframe):
    for column in dframe:
        enc = LabelEncoder()
        if(column not in ['age','education.num','fnlwgt','capital.gain','capital.loss','hours.per.week']):
            dframe[column] = enc.fit_transform(dframe[column])
    return dframe

people = preprocess_features(data)

Importing the data points from our previous PCA analysis in homework 4:

In [ ]:
# get PCA results from previous homework via pickle file "pca_coordinates.p")
from urllib.request import urlopen
pca_coordinates = pickle.load(urlopen('http://www.columbia.edu/~chw2/Courses/PPF/Homeworks/2021/Homework5/pca_coordinates.p'))

In [ ]:
len(pca_coordinates)

Now we again have a dataframe `people` that has 15 data points for 32,561 people (just like in the previous homework). We also have the PCA results for the last 1500 people from our previous homework in the list `pca_coordinates`. Note that the order of the PCA coordinates is also the order of the last 1500 people in `people`. Plotting our PCA results, we have:

In [ ]:
# Plot PCA results from last homework for
X = []
Y = []
for person in pca_coordinates:
    X.append(person[0])
    Y.append(person[1])
plt.figure(figsize=(6, 6))
plt.title('PCA for last 1500 people', fontsize=15)
plt.xlabel('PC 1', fontsize=10)
plt.ylabel('PC 2', fontsize=10)
plt.scatter(X,Y, alpha = .4, s = 40.)
plt.show()

Also, in case you were curious, here's an image of PCA results performed on the entire dataset `people`, with the datapoint opacity (i.e., alpha) set very low so we can easily see how the data points are distributed.
<img src="pca_all.png">
This gives us a hutch about both how many clusters we might want to look for as well as where these clusters might be located. We're just going to use the last 1500 people for our analysis in question 1.


### question 1 [20 points]  | clustering people
Now we're going to use K-means in an attempt to cluster the people in our dataset `pca_coordinates` without explicitly defining what these clusters will be at the outset (i.e., we're doing unsupervised learning).

ASIDE: If our discussion of K-means in lab 10a left you scratching your head, a helpful tutorial on K-means with python code here can be found here: https://mubaris.com/posts/kmeans-clustering/ . After picking the number of cluster center locations, K-means basically repeats the following two steps over and over: (1) reassign points according to which cluster center they are closest to and (2) readjusts the cluster center based on the mean distance of the data points in each cluster. The K-means procedure is completed once the cluster means and labels stop changing after number of iterations.

#### To see how this is implemented in scikit learn, consider the following example. Let's take `X_example` data from the two columns "age" and "hours worked" in `people` and perform K-means on it.  

In [ ]:
# Gather X_example data from people dataframe
# You won't need to use this cell for question 1 as we already have that data in `pca_coordinates` list

# this takes the columns "sex" and "marital.status" and
# combines them into a list of paired values that scikit-learn expects
X_example = list(zip(people["age"][0:500].values, people["hours.per.week"][0:500].values))

In [ ]:
# Perform K_means on X_example

# define number of clusters
K = 2 # change K to whatever number of clusters you want
kmeans_model = KMeans(n_clusters=K, n_init='auto')

# perform K-means on X_example data
kmeans = kmeans_model.fit(X_example)

# Get cluster labels for each data point in X_example
# where each cluster is denoted by a number. For six
# clusters, the cluster labels = [0, 1, 2, 3, 4, 5].
labels = kmeans.predict(X_example)

# Get cluster center positions for each cluster.
# This will just be the x,y coordinates for X_example.
centers = kmeans.cluster_centers_

We can plot X_example data and the cluster centroids as follows:

In [ ]:
print(centers) # will be pairs of x,y coordinates

We can plot the centers of the clusters (denoted by stars) and the X_example data color-coded by cluster like this:

In [ ]:
plt.figure(figsize=(12, 12))
plt.title('K-means on 500 people for age v hours worked', fontsize=15)
plt.xlabel('age', fontsize=10)
plt.ylabel('hours worked', fontsize=10)

## THIS CODE IS USED TO PRODUCE "INDEX LABELS"
## uncomment the "#" below to make an index label for each data point
# ax = plt.subplot(111) #used to make indices
# for text in range(0, len(X_example)):
#    ax.annotate(text, xy=(X_example[text]), xycoords='data', xytext=(-30, -30), textcoords='offset points')

# plot data
plt.scatter([x[0] for x in X_example], [y[1] for y in X_example], c=labels, alpha = 0.6)
plt.scatter([c_x[0] for c_x in centers], [c_y[1] for c_y in centers], marker='*', c='#050505', s=500)
plt.show()

What are we to make of this? It's not obvious what two clusters we would expect using the "age" and "hours worked per week" data for 500 people. Since we picked 2 clusters, k-means effectively clustered our data into two halves. But you can easily change the number of clusters `K` and rerun k-means to see what different clusters you get. The stars, as already mentioned, denote the location of the centers of the clusters.

#### Your turn!

#### 1. Using the example above as a guide, perform K-means on our `pca_coordinates` data. (Hint: you'll use the list  `pca_coordinates` in place of `X_example` list above.)
1a. Pick how many clusters `K` to use by examining plots "PCA for last 1500 people" and the "2 component PCA for Similarity" plots above. (Going off the PCA plots above, likely anywhere between 5 and 8 clusters might give good clusters.)

#### 2. Graph the K-means clusters using the example code above. Uncomment the code that annotates the datapoints with their corresponding indices.

In [ ]:
# 1. WRITE K-MEANS CODE HERE


In [ ]:
# 2. WRITE GRAPHING CODE HERE


## question 2 [30 points] | Writing fictions with data
You've performed K-means on a PCA plot of the last 1500 people. Now it's time to use this data to make claims about the world.

Imagine you're working in an advertising firm. Pick one cluster and examine what all the people around cluster have in common. They probably won't share just one attribute, but several similar features. Note which cluster you're looking at and *produce a story* about what life is like for these people. Be creative and don't be afraid to make inferences that are not contained by the data, as long as the data doesn't contradict your story.  Are they old? Young? How much do they work? Men? Women? Married? *Use evidence from the data to support your story.* [max 250 words, min 150 words]

If you used the example code above, the coordinates of the cluster centers are stored in the list `centers`. To figure out what data points (i.e., people) are around a cluster, uncomment and use the code for "INDEX LABELS" above. This will attach a label to each data point. Once you have labels, you can zoom in on the section around the cluster to more easily read the index labels.

Because we took the last 1500 people to produce our PCA coordinates and because our data set has 32561 people (with each row being one person), datapoint/person "1" will be the 31061st row in our dataframe `people`. Likewise datapoint/person "2" will be the 31062nd person, etc. The datapoint/person "1500" will be the last row in the data set. You can use the usual panda tricks on the dataframe `people` to see what these people have in common, and you can use this to infer/imagine what the cluster of people represents.

Provide your story of the people in the cluster your choosing below:

In [ ]:
# Run this code to get raw data associated with each person in a cluster
adult = pd.read_csv('https://raw.githubusercontent.com/data-ppf/data-ppf.github.io/main/dat/adult.csv')
from pandas import DataFrame
my_people_indexes = [] # ENTER CODE HERE: CHOOSE SOME INDICIES IN A CLUSTER TO PUT IN THIS LIST

for i in range(0, len(my_people_indexes)):
  my_people_indexes[i] = my_people_indexes[i] + 31060

my_people = adult.iloc[my_people_indexes, :]

my_people

Put your story here...

.

.

.

.

.

.


## Question 3: Undermining privacy [40 points]

In this section, you're going to undertake something like the deanonymizing process we undertook during lab, but with your old friend the adult data set, now with (fake) personal identification--fake Social security numbers, etc. And then you'll do some work to make the data set less prone to abuse

Let's load the files.

In [ ]:
# getting the data
import pandas as pd
import numpy as np

adult = pd.read_csv('https://raw.githubusercontent.com/data-ppf/data-ppf.github.io/main/dat/adult.csv')
adult_pii = pd.read_csv('https://raw.githubusercontent.com/data-ppf/data-ppf.github.io/main/dat/adult_pii.csv')

In [ ]:
adult.head()

In [ ]:
adult_pii.head()

Our goal is to deanonymize information in the adult data set by linking it to the highly private information in `adult_pii`.

You task reidentify 4 people. That is, show which rows the names in the `adult` data set correspond to 4 people.

HINT 1: Use techniques from the lab to help you. In the lab, we used `merge` on two large datasets; you can use merge with a single line of code using the names of the dataframes.

HINT 2: Which columns are shared between both dataframes? Pandas `merge` lets you join on multiple columns if you pass a list into the `on` parameter

In [ ]:
# ENTER CODE HERE


# steps toward making a set k-anonymous


Now you need to generalize your data.

Age is very granular. Let's make it much less so.

Write a function that converts the age of anyone greater than or equal to 50 to "Ok, boomer" and that age of anyone younger than 50 to "Avocado Toast".

Use "map" to apply your function to every value in the age column.


In [ ]:
def age_generalization(age):
    if ## insert your code
        return ###
    else:
        return ####

#then apply this function to every row using the map function--a very powerful tool

data.age = data.age.map(age_generalization)
display(data)